# Optimisation for Machine Learning

October 04, 2023

### Logistic
Contact: [Clement Royer](mailto:clement.royer@lamsade.dauphine.fr)
Lecture's web: [URL](https://www.lamsade.dauphine.fr/%7Ecroyer/teachOAA.html)
Examen: 60% (2h), dated December 13, 2023 10:00 AM - 12:00 PM
Project: 40%, during from October 6, 2023 to December 23, 2023

In [3]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('default')
plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rc('font', size=18)
plt.rc('axes', titlesize=18)
plt.rc('axes', labelsize=18)
plt.rc('xtick', labelsize=18)
plt.rc('ytick', labelsize=18)
plt.rc('legend', fontsize=18)
plt.rc('lines', markersize=10)

**Today**: More on Gradient Descent and Non-Convexity
**Tomorrow**: Regularization and Proximal Gradient

**Problem**: $\min_{w \in \mathbb{R}^d} f(w)$ where $f: \mathbb{R}^d \rightarrow \mathbb{R}$ is convex and differentiable.

Gradient descent iteration: $\forall k \geq 0, w^{k+1} = w^k - \alpha_k \nabla f(w^k)$ where $\alpha_k > 0$ is the step size.

#### Choosing the step size $\alpha_k$ (a.k.a learning rate in ML): 

**Recall**: When $f in \mathcal{C}^{1, 1}_L$ (i.e. $\mathcal{C}^{1} + \nabla f$ is $L$-Lipschitz continuous), $\alpha_k = \frac{1}{L}$ is the good step size choice because it guarantees that $f(w^{k+1}) \leq f(w^k) - \frac{1}{2L} \|\nabla f(w^k)\|^2$. (Any $\alpha_k \in (0, \frac{2}{L})$ is also good.)

In practice, the value of $L$ could be too expensive to compute/unknown/not exist.

In this case, there are 3 main strategies to choose $\alpha_k$:
- **Constant step size**: $\alpha_k = \alpha$ for all $k \geq 0$ (e.g. $\alpha = 0.1, 0.01, 0.001$). For a sufficiently small this choice leads to convergence. For $\mathcal{C}^{1, 1}_L$ functions, $\alpha \leq \frac{2}{L}$ works. This is very popular in practice but difficult to calibrate in advance.
- **Diminishing step size**: $\alpha_k \rightarrow 0$ as $k \rightarrow \infty$. For example, $\alpha_k = \frac{\alpha}{k+1}$ or $\alpha_k = \frac{\alpha}{\sqrt{k+1}}$. For $k$ large enough, $\alpha_k \leq \frac{2}{L}$ and convergence is guaranteed for $\mathcal{C}^{1, 1}_L$ functions. Less popular than constant step size yet but useful in stochastic settings.
- **Adaptive step size**: (Learning rate scheduling) Idea is that $\alpha_k$ is chosen according to $f$ and $w^k$, $\nabla f(w^k)$, $\nabla f(w^{k-1})$, ... (e.g. $\alpha_k = \frac{\alpha}{\|\nabla f(w^k)\|}$). typically done via a line search. The GD step has the form $\underbrace{w^k - \alpha \nabla f(w^k)}_{\text{function of } \alpha}$
. We would like the best possible value of $\alpha$ in term of function value $f(w_k - \alpha \nabla f(w^k))$ as small as possible.

Exact line search: $\alpha_k = \arg\min_{\alpha \geq 0} f(w^k - \alpha \nabla f(w^k))$. This is expensive to compute. In practice, we replace by an approximate line search.

**Anmijo backtracking line search**
Start with $\alpha_k = \overline{\alpha}$ (e.g. $\overline{\alpha} = 1$). While $f(w^k - \alpha_k \nabla f(w^k)) > f(w^k) - C \alpha_k \|\nabla f(w^k)\|^2$ (e.g. $C = 10^{-4}$), update $\alpha_k = \beta \alpha_k$ (e.g. $\beta = 0.5$). Stop when $f(w^k - \alpha_k \nabla f(w^k)) \leq f(w^k) - C \alpha_k \|\nabla f(w^k)\|^2$.
Process based on a sufficient decrease condition. (See [Nocedal and Wright, Numerical Optimization, 2nd edition, 2006, p. 37](https://link.springer.com/book/10.1007/978-0-387-40065-5)). The condition will be violated $(f(w^k - \alpha_k \nabla f(w^k)) \leq f(w^k) - C \alpha_k \|\nabla f(w^k)\|^2)$ when $\alpha_k$ is small enough. The condition is satisfied when $\alpha_k$ is large enough.

If $f \in \mathcal{C}^{1, 1}_L$ and $C = \frac{1}{2}$, then $\alpha_k \leq \frac{2}{L}$ and convergence is guaranteed.
$\Longrightarrow$ for $\alpha_k = \frac{1}{L}$, we recover the guarantee of the constant step size $f(w_k - \alpha_k \nabla f(w_k)) \leq f(w_k) - \frac{1}{2L} \|\nabla f(w_k)\|^2$.

This strategy is more expensive than using constant step size or decreasing step size because it requires function evaluations $f(w^k) + f(w^k - \alpha_k \nabla f(w^k))$ for every $\alpha_k$ used in the backtracking line search. In ML (mostly in deep learning) this is often considered as prohibitive.